In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')  # use this if you are using google colab

In [1]:
from sklearn.cluster import KMeans
import pickle, os
from scipy.spatial import distance
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [2]:
def load_dataset(name_file):

    desired_directory = '.' # Replace with your desired directory path

    file_path = os.path.join(desired_directory, name_file) 

    with open(file_path, 'rb') as f:
        data_dict = pickle.load(f)

    return data_dict


In [3]:
loaders_dict = load_dataset(f"dataset-flowers102-features.pkl")
x_train = loaders_dict["x_train"]
x_test = loaders_dict["x_test"]
y_train = loaders_dict["y_train"]
y_test = loaders_dict["y_test"]
print(f"x_train:{x_train.shape}, y_train:{y_train.shape}")
print(f"x_test:{x_test.shape}, y_test:{y_test.shape}")


x_train:(4094, 512), y_train:(4094,)
x_test:(4095, 512), y_test:(4095,)


## clustering images

In [32]:
k_number = 50
kmeans = KMeans(n_clusters=k_number)
clusters = kmeans.fit_predict(x_train)
centroids = kmeans.cluster_centers_

c:\Users\ewint\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [9]:
# centroid_distances = [distance.euclidean(x_test[0], centroid) for centroid in centroids]
# centroid_distances

## find nearest clusters neighbors

### find nearest clusters and their indices

In [66]:
def find_nearest_clusters_neighbors(x_train, x_test, k):
    knn_classifier = KNeighborsClassifier(n_neighbors=k)
    knn_classifier.fit(x_train, range(len(x_train)))
    nearest_indices = knn_classifier.kneighbors([x_test], n_neighbors=k, return_distance=False)[0]
    nearest_neighbors = x_train[nearest_indices]
  
    return nearest_indices, nearest_neighbors

In [67]:
n_nearest_neighbors = 3
nearest_clusters_indices, nearest_clusters_neighbors = find_nearest_clusters_neighbors(centroids, x_test[0], k=n_nearest_neighbors)

In [68]:
print(f"clusters Indices:")
print(nearest_clusters_indices)
print(f"nearest clusters neighbors:")
print(nearest_clusters_neighbors)

clusters Indices:
[25  9 36]
nearest clusters neighbors:
[[0.40670767 0.7701734  1.337215   ... 1.7432996  0.27299303 1.8507631 ]
 [0.07480335 0.6644728  1.3205874  ... 1.0844386  1.2337798  2.0579867 ]
 [0.14745116 0.30148768 1.1407182  ... 0.5903618  2.7259123  1.6983356 ]]


### get the data of nearest clusters

In [69]:
def gather_clusters_data(k_number, clusters):
    clusters_data = {i: [] for i in range(k_number)}
    for i, label in enumerate(clusters):
        clusters_data[label].append(x_train[i])
    return clusters_data

In [70]:
clusters_data = gather_clusters_data(k_number, clusters)

In [82]:
def get_nearest_clusters_data(nearest_clusters_indices, clusters_data):
    # Access data points in the cluster with centroid index 'nearest_clusters_indices'
    data_in_nearest_clusters = []
    for index in nearest_clusters_indices:
        data_in_nearest_clusters.extend(clusters_data[index])
    return(data_in_nearest_clusters)

In [83]:
data_in_nearest_clusters = get_nearest_clusters_data(nearest_clusters_indices, clusters_data)

## classify using data in nearest clusters

In [ ]:
# def classify_knn(x_test, x_train, y_test, y_train, k):
#     knn_classifier = KNeighborsClassifier(n_neighbors=k)
#     knn_classifier.fit(x_train, y_train)
#     y_pred = knn_classifier.predict(x_test)
#     accuracy = accuracy_score(y_test, y_pred)
    
#     return y_pred, accuracy

In [ ]:
# k = 3
# predictions, accuracy = classify_knn(x_train, y_train, x_test, y_test, k=k)
# print(f"Predicted labels: {predictions}")
# print(f"Accuracy: {accuracy * 100:.2f}%")